# Triangulation

In [1]:
import os
import pickle
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from time import time
from lib import misc, utils, app
from lib.calib import triangulate_points_fisheye
import cv2
# import json
# import colorsys

plt.style.use(os.path.join('..', 'configs', 'mplstyle.yaml'))

%load_ext autoreload
%autoreload 2
%matplotlib auto

ROOT_DATA_DIR = os.path.join("/Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main","data")

Using matplotlib backend: MacOSX


# Ground Truth Params
Define the params in the cell below. Thereafter, run all cells

In [2]:
DATA_DIR = os.path.join(ROOT_DATA_DIR, "2021_10_07", "Baboon1", "walk")

start_frame = 24420
end_frame = 24469


# Ground Truth

In [3]:
assert os.path.exists(DATA_DIR)
OUT_DIR = os.path.join(DATA_DIR, 'ground_truth')
os.makedirs(OUT_DIR, exist_ok=True)


k_arr, d_arr, r_arr, t_arr, cam_res, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR, verbose=False)


dlc_points_fpaths = sorted(glob(os.path.join(OUT_DIR,'csv', '*.csv')))

start_frame -= 1 # 0 based indexing
assert start_frame >= 0
N = end_frame-start_frame

# Load Measurement Data (pixels, likelihood)
dfs = []
for path in dlc_points_fpaths:
    dlc_df = pd.read_csv(path, sep=";")
   # dlc_df = dlc_df.droplevel([0], axis=1).swaplevel(0,1,axis=1).T.unstack().T.reset_index().rename({'level_0':'frame'}, axis=1)
    dlc_df.columns.name = ''
    dfs.append(dlc_df)
#create new dataframe
dlc_df = pd.DataFrame(columns=['frame', 'camera', 'marker', 'x', 'y', 'likelihood'])
for i, df in enumerate(dfs):
    df['camera'] = i
    df.rename(columns={'bodyparts':'marker'}, inplace=True)
    dlc_df = pd.concat([dlc_df, df], sort=True, ignore_index=True)

dlc_df = dlc_df[['frame', 'camera', 'marker', 'x', 'y', 'likelihood']]

print(dlc_df)

points_2d_df = dlc_df

#points_2d_df = points_2d_df[points_2d_df["frame"].between(start_frame, end_frame-1)]

assert len(k_arr) == 2

points_3d_df = utils.get_pairwise_3d_points_from_df(
    points_2d_df,
    k_arr, d_arr.reshape((-1,4)), r_arr, t_arr,
    triangulate_points_fisheye
)

points_3d_df['point_index'] = points_3d_df.index

print(points_3d_df)

     frame camera      marker       x      y likelihood
0    24420      0  left_ankle   901.0  773.0          1
1    24420      0  left_elbow  1022.0  751.0          1
2    24420      0    left_eye     NaN    NaN          1
3    24420      0    left_hip     NaN    NaN          1
4    24420      0   left_knee     NaN    NaN          1
..     ...    ...         ...     ...    ...        ...
139  24468      1     spine_4   830.0  693.0          1
140  24468      1      tail_1   798.0  690.0          1
141  24468      1      tail_2   786.0  714.0          1
142  24468      1   tail_base   811.0  698.0          1
143  24468      1    tail_end   783.0  741.0          1

[144 rows x 6 columns]

    frame      marker         x         y         z  point_index
0   24420  left_ankle -0.595706  5.478000 -0.691443            0
1   24420  left_elbow  0.065753  5.110207 -0.526833            1
2   24420    left_eye       NaN       NaN       NaN            2
3   24420    left_hip       NaN       NaN  

# Save triangulation results

In [4]:
markers = misc.get_markers()

positions = np.full((N, len(markers), 3), np.nan)
for i, marker in enumerate(markers):
    marker_pts = points_3d_df[points_3d_df["marker"]==marker][["frame", "x", "y", "z"]].values
    for frame, *pt_3d in marker_pts:
        positions[int(frame)-start_frame, i] = pt_3d

app.save_tri(positions, OUT_DIR, scene_fpath, start_frame, 0)

Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/ground_truth/tri.pickle
Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/ground_truth/tri.mat
Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/ground_truth/cam*_ground_truth.h5
Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/ground_truth/cam*_ground_truth.csv

Saving labeled videos...


  0%|          | 2/25627 [00:00<23:03, 18.53it/s]

Loading cam2 and data.

Duration of video: 427.12 s, recorded with 60.0 fps!
Total frames: 25627 with frame dimensions: 1920 x 1080
Generating frames and creating video...
Loading cam1 and data.

Duration of video: 427.12 s, recorded with 60.0 fps!
Total frames: 25627 with frame dimensions: 1920 x 1080
Generating frames and creating video...


100%|██████████| 25627/25627 [08:42<00:00, 49.06it/s]



Done!



# Plot the baboon!

In [5]:
data_fpath = os.path.join(OUT_DIR, 'tri.pickle')
app.plot_baboon_reconstruction(data_fpath, dark_mode=True)

Loaded extrinsics from /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/extrinsic_calib/2_cam_scene_sba.json



qt.qpa.window: <QNSWindow: 0x7fc2d32e1860; contentView=<QNSView: 0x7fc2d32e0700; QCocoaWindow(0x7fc2d32e05f0, window=QWidgetWindow(0x7fc2d32e0100, name="QGLWidgetClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.
/Users/matthewterblanche/opt/anaconda3/lib/python3.8/site-packages/pyqtgraph/graphicsItems/PlotCurveItem.py:153: RuntimeWarning: All-NaN slice encountered
  b = (np.nanmin(d), np.nanmax(d))
